In [192]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pwaic-knn/training.csv
/kaggle/input/pwaic-knn/testing.csv


In [193]:
import time
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

 
train= pd.read_csv('../input/pwaic-knn/training.csv')
test= pd.read_csv('../input/pwaic-knn/testing.csv')


def get_score(model , X_data , Y_data , test_ratio ,random_state ):
    X_train, X_test, Y_train, Y_test = train_test_split( X_data, Y_data, test_size=test_ratio , random_state=random_state) 
    model.fit(X_train ,Y_train)
    return model.score(X_test,Y_test)


def fill_missing(data):
    for col in data.columns:
        data[col] = data[col].fillna(data[col].median())
 
 
print(train.head())
print(test.head())

 
 


   sepal_length  sepal_width  petal_length  petal_width            label
0           7.7          3.8           6.7          2.2   Iris-virginica
1           5.1          2.5           3.0          1.1  Iris-versicolor
2           7.1          3.0           5.9          2.1   Iris-virginica
3           6.1          3.0           4.9          1.8   Iris-virginica
4           4.7          3.2           1.3          0.2      Iris-setosa
   sepal_length  sepal_width  petal_length  petal_width
0           6.9          3.1           5.1          2.3
1           4.5          2.3           1.3          0.3
2           5.0          2.0           3.5          1.0
3           6.2          3.4           5.4          2.3
4           6.5          3.0           5.2          2.0


In [194]:
le  = LabelEncoder()
le.fit(train['label'])
label = le.transform(train['label'])
train.drop(['label'] , axis =1)
train['label'] = label
fill_missing(train)

for col in train.columns:
    if col=='label':
        continue
    train[col] = train[col]/train[col].max()
    test[col] = test[col]/test[col].max()

print(train.head())
print(test.head())
X_data = train.drop(['label'] ,axis=1)
Y_data = train['label']
X_train, X_test, Y_train, Y_test = train_test_split( X_data, Y_data, test_size=0.20, random_state=0) 


 


   sepal_length  sepal_width  petal_length  petal_width  label
0      1.000000     0.863636      0.971014         0.88      2
1      0.662338     0.568182      0.434783         0.44      1
2      0.922078     0.681818      0.855072         0.84      2
3      0.792208     0.681818      0.710145         0.72      2
4      0.610390     0.727273      0.188406         0.08      0
   sepal_length  sepal_width  petal_length  petal_width
0      0.873418     0.738095      0.796875         0.92
1      0.569620     0.547619      0.203125         0.12
2      0.632911     0.476190      0.546875         0.40
3      0.784810     0.809524      0.843750         0.92
4      0.822785     0.714286      0.812500         0.80


In [195]:

#classification models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier

modelNames=['DecisionTreeClassifier', 'RandomForestClassifier',
            'KNeighborsClassifier', 'GaussianNB', 
            'LogisticRegression', 'svm', 'MLPClassifier']
model = [DecisionTreeClassifier() , 
        RandomForestClassifier(),
       KNeighborsClassifier() ,  
       GaussianNB(),
       LogisticRegression(),
       svm.LinearSVC(),
        MLPClassifier(random_state=1, max_iter=300)
        ]

#evaluation
from sklearn.metrics import accuracy_score , confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest, chi2 ,  RFE  
from sklearn.model_selection import  cross_val_score


print('simple accuracy = ' ,get_score(model[2] ,X_data, Y_data  ,.20  , 0))

    
    

simple accuracy =  0.9444444444444444


In [196]:
for i in range(1,len(train.columns)):
    sel_f = SelectKBest(chi2, k = i).fit(X_data,Y_data)
    sel_bool = sel_f.get_support()
#     print(sel_bool)
    X  = X_data[X_data.columns[sel_bool]] 
    print(X.head())
    print(get_score(model[2] , X  , Y_data, .20 , 0 ))





   petal_width
0         0.88
1         0.44
2         0.84
3         0.72
4         0.08
0.9444444444444444
   petal_length  petal_width
0      0.971014         0.88
1      0.434783         0.44
2      0.855072         0.84
3      0.710145         0.72
4      0.188406         0.08
0.9444444444444444
   sepal_length  petal_length  petal_width
0      1.000000      0.971014         0.88
1      0.662338      0.434783         0.44
2      0.922078      0.855072         0.84
3      0.792208      0.710145         0.72
4      0.610390      0.188406         0.08
0.9444444444444444
   sepal_length  sepal_width  petal_length  petal_width
0      1.000000     0.863636      0.971014         0.88
1      0.662338     0.568182      0.434783         0.44
2      0.922078     0.681818      0.855072         0.84
3      0.792208     0.681818      0.710145         0.72
4      0.610390     0.727273      0.188406         0.08
0.9444444444444444


In [197]:
#parameter tuning
Y_data.head()
knn  =  KNeighborsClassifier(n_neighbors=10)

# print(get_score(knn,X_data , Y_data   , .2   , 0  ))
# print(get_score(knn,X_data , Y_data   , .1   , 1  ))
# print(get_score(knn,X_data , Y_data   , .5   , 2  ))




In [198]:
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV

leaf_size = list(range(20,40))
n_neighbors = list(range(1,15))
p=[1,2,3]

param_random = {
    'leaf_size':leaf_size, 
    'n_neighbors': n_neighbors , 
    'p':p  
}


In [199]:
random_cv = RandomizedSearchCV(estimator=KNeighborsClassifier() , param_distributions=param_random  ,
                              cv = 4 ,verbose=2 ,scoring  = 'accuracy' ,n_iter=2 , n_jobs=-1 )
random_cv.fit(X_data ,Y_data)

Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.1s finished


RandomizedSearchCV(cv=4, estimator=KNeighborsClassifier(), n_iter=2, n_jobs=-1,
                   param_distributions={'leaf_size': [20, 21, 22, 23, 24, 25,
                                                      26, 27, 28, 29, 30, 31,
                                                      32, 33, 34, 35, 36, 37,
                                                      38, 39],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14],
                                        'p': [1, 2, 3]},
                   scoring='accuracy', verbose=2)

In [200]:
param_grid = {
    'leaf_size':[random_cv.best_params_['leaf_size']-1 , 
                random_cv.best_params_['leaf_size'] , 
                random_cv.best_params_['leaf_size']+1], 
    'n_neighbors':  [random_cv.best_params_['n_neighbors']-2 , 
                random_cv.best_params_['n_neighbors']-1 , 
                random_cv.best_params_['n_neighbors'] , 
                 random_cv.best_params_['n_neighbors']+1 ,
                    random_cv.best_params_['n_neighbors']+2 ] , 
    'p':[random_cv.best_params_['p']-1 ,
        random_cv.best_params_['p'] ,
        random_cv.best_params_['p']+1]  
}
grid_cv = GridSearchCV( estimator=KNeighborsClassifier() , param_grid=param_grid , verbose=2 ,n_jobs=-1 ,
                      cv =4 , scoring='accuracy')
grid_cv.fit(X_data , Y_data)


Fitting 4 folds for each of 45 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 107 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    0.7s finished


GridSearchCV(cv=4, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'leaf_size': [21, 22, 23],
                         'n_neighbors': [5, 6, 7, 8, 9], 'p': [0, 1, 2]},
             scoring='accuracy', verbose=2)

In [202]:
knn = grid_cv.best_estimator_

knn.fit(X_data , Y_data)
ans = knn.predict(test)
ans = le.inverse_transform(ans) 

ans =  pd.DataFrame({
    'id': list( range(1,61,1)),
    'label':ans
})

ans.to_csv('PWAIC KNN Contest! vikassao.csv' ,index =False)
ans.head()


,id,label
0,1,Iris-virginica
1,2,Iris-setosa
2,3,Iris-versicolor
3,4,Iris-virginica
4,5,Iris-virginica
